In [1]:
import pyranges as pr
import numpy as np
import pandas as pd
from Bio.Seq import Seq
from Bio.Seq import MutableSeq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio import motifs
from papa_helpers import get_terminal_regions
import os
import sys
import random

## Reference transcripts


In [2]:
# A plus strand transcript
test_ref_tr1 = {"Chromosome": ["chr1"]*5,
                "Start": [100,100,1000,2000,3000],
                "End": [3400,300,1200,2200,3400],
                "Strand": ["+"]*5,
                "Feature": ["transcript"] +["exon"]*4,
                "gene_id": ["id_ref_gene_1"]*5,
                "transcript_id": ["ref_g1_tr_1"] * 5,
                "gene_name": ["ref_gene_1"]*5,
                "exon_number": [None,1,2,3,4],
                "gene_type": ["protein_coding"] * 5}


pr.from_dict(test_ref_tr1)


,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr1,100,3400,+,transcript,id_ref_gene_1,ref_g1_tr_1,ref_gene_1,NaN,protein_coding
1,chr1,100,300,+,exon,id_ref_gene_1,ref_g1_tr_1,ref_gene_1,1.0,protein_coding
2,chr1,1000,1200,+,exon,id_ref_gene_1,ref_g1_tr_1,ref_gene_1,2.0,protein_coding
3,chr1,2000,2200,+,exon,id_ref_gene_1,ref_g1_tr_1,ref_gene_1,3.0,protein_coding
4,chr1,3000,3400,+,exon,id_ref_gene_1,ref_g1_tr_1,ref_gene_1,4.0,protein_coding


In [3]:
# A minus strand transcript
test_ref_tr2 = {"Chromosome": ["chr2"]*4,
                "Start": [100,100,700,1000],
                "End": [1200,200,800,1200],
                "Strand": ["-"]*4,
                "Feature": ["transcript"] + ["exon"]*3,
                "gene_id": ["id_ref_gene_2"]*4,
                "transcript_id": ["ref_g2_tr_1"]*4,
                "gene_name": ["ref_gene_2"]*4,
                "exon_number": [None,3,2,1],
                "gene_type": ["protein_coding"] * 4
               }


test_ref = pr.concat([pr.from_dict(test_ref_tr1), pr.from_dict(test_ref_tr2)])
test_ref

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr1,100,3400,+,transcript,id_ref_gene_1,ref_g1_tr_1,ref_gene_1,NaN,protein_coding
1,chr1,100,300,+,exon,id_ref_gene_1,ref_g1_tr_1,ref_gene_1,1.0,protein_coding
2,chr1,1000,1200,+,exon,id_ref_gene_1,ref_g1_tr_1,ref_gene_1,2.0,protein_coding
3,chr1,2000,2200,+,exon,id_ref_gene_1,ref_g1_tr_1,ref_gene_1,3.0,protein_coding
4,chr1,3000,3400,+,exon,id_ref_gene_1,ref_g1_tr_1,ref_gene_1,4.0,protein_coding
5,chr2,100,1200,-,transcript,id_ref_gene_2,ref_g2_tr_1,ref_gene_2,NaN,protein_coding
6,chr2,100,200,-,exon,id_ref_gene_2,ref_g2_tr_1,ref_gene_2,3.0,protein_coding
7,chr2,700,800,-,exon,id_ref_gene_2,ref_g2_tr_1,ref_gene_2,2.0,protein_coding
8,chr2,1000,1200,-,exon,id_ref_gene_2,ref_g2_tr_1,ref_gene_2,1.0,protein_coding


## Test novel transcripts


In [4]:
# Collecting 3'ends for a dummy PolyASite atlas
pas_ends = {"Chromosome": [],
           "Start": [],
           "End": [],
           "Strand": []}

# Collect test novel transcripts
test_novel_trs = []


pas_ends

{'Chromosome': [], 'Start': [], 'End': [], 'Strand': []}

In [5]:
def update_pas_ends(pas_dict, chrom, start, end, strand):
    '''
    '''
    
    pas_d = pas_dict.copy()
    
    pas_d["Chromosome"].append(chrom)
    pas_d["Start"].append(start)
    pas_d["End"].append(end)
    pas_d["Strand"].append(strand)
    
    return pas_d
    

In [6]:
# Now make test novel transcripts to cover my test cases

# 1. Novel last exons in first intron of annotated transcript (e.g. STMN2)
# For these to pass, they should share an identical 3'end with a first exon of a known transcript

test_novel_fi = {"Chromosome": ["chr1"]*6,
                "Start": [100] + [100,500] + [100] + [100,500],
                "End": [700,300,700,700,350,700],
                "Strand": ["+"]*6,
                "Feature": ["transcript"] + ["exon"]*2 + ["transcript"] + ["exon"]*2,
                "gene_id": ["id_nov_gene_1"]*6,
                "transcript_id": ["nov_gene_1_tx_fi_p"]*3 + ["nov_gene_1_tx_fi_f"]*3,
                 "gene_name": ["nov_gene_1"]*6,
                "exon_number": [None,1,2,None,1,2],
                 "gene_type": ["protein_coding"] * 6
               }

test_novel_trs.append(pr.from_dict(test_novel_fi))
update_pas_ends(pas_ends, "chr1", 699, 700, "+")
pr.from_dict(test_novel_fi)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr1,100,700,+,transcript,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,NaN,protein_coding
1,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,1.0,protein_coding
2,chr1,500,700,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,2.0,protein_coding
3,chr1,100,700,+,transcript,id_nov_gene_1,nov_gene_1_tx_fi_f,nov_gene_1,NaN,protein_coding
4,chr1,100,350,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_f,nov_gene_1,1.0,protein_coding
5,chr1,500,700,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_f,nov_gene_1,2.0,protein_coding


In [7]:


pas_ends

{'Chromosome': ['chr1'], 'Start': [699], 'End': [700], 'Strand': ['+']}

In [8]:
#1b. First intron extensions
# 1 should fall within x bp of the 5'end of the annotated first exon (to account for diffs in annotation)
# To fail should fall outside x bp (e.g. 500bp away) - easier to do this on minus strand as starts at 1200 (1999)

test_novel_fi_ext = {"Chromosome": ["chr2"]*4,
                     "Start": [880,880] + [880,880],
                "End": [1200,1200] + [1500,1500],
                "Strand": ["-"]*4,
                "Feature": ["transcript", "exon"]*2,
                "gene_id": ["id_nov_gene_2"]*4,
                "transcript_id": ["nov_gene_2_tx_fi_ext_p"]*2 + ["nov_gene_2_tx_fi_ext_f"]*2,
                     "gene_name": ["nov_gene_2"]*4,
                     "exon_number": [None,1,None,1],
                     "gene_type": ["protein_coding"] * 4
                    }

test_novel_trs.append(pr.from_dict(test_novel_fi_ext))
update_pas_ends(pas_ends, "chr2", 880, 881, "-")
pr.from_dict(test_novel_fi_ext)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr2,880,1200,-,transcript,id_nov_gene_2,nov_gene_2_tx_fi_ext_p,nov_gene_2,NaN,protein_coding
1,chr2,880,1200,-,exon,id_nov_gene_2,nov_gene_2_tx_fi_ext_p,nov_gene_2,1.0,protein_coding
2,chr2,880,1500,-,transcript,id_nov_gene_2,nov_gene_2_tx_fi_ext_f,nov_gene_2,NaN,protein_coding
3,chr2,880,1500,-,exon,id_nov_gene_2,nov_gene_2_tx_fi_ext_f,nov_gene_2,1.0,protein_coding


In [9]:
pas_ends

{'Chromosome': ['chr1', 'chr2'],
 'Start': [699, 880],
 'End': [700, 881],
 'Strand': ['+', '-']}

In [10]:
# 2. Internal intron, spliced in last exon (e.g. ONECUT1)
# For these to pass, the 5'end of the novel last exon must exactly match a reference intron
# Also include an event that passes the 5'end filter but won't have a nearby annotated pas

test_novel_mi_s = {"Chromosome": ["chr1"]*12,
                "Start": [100,100,1000,1400] + [100,100,1000,1400] + [100, 100, 1000, 1750],
                "End": [1600,300,1200,1600] + [1600,300,1300,1600] + [1850, 300, 1200, 1850],
                "Strand": ["+"]*12,
                "Feature": (["transcript"] + ["exon"]*3)*3,
                "gene_id": ["id_nov_gene_1"]*12,
                "transcript_id": ["nov_gene_1_tx_mi_s_p"]*4 + ["nov_gene_1_tx_mi_s_f"]*4 + ["nov_gene_1_tx_mi_s_no_pas_no_motif_f"]*4,
                 "gene_name": ["nov_gene_1"] * 12,
                "exon_number": [None,1,2,3,None,1,2,3, None,1,2,3],
                 "gene_type": ["protein_coding"] * 12
               }
test_novel_trs.append(pr.from_dict(test_novel_mi_s))
update_pas_ends(pas_ends, "chr1", 1599, 1600, "+")
pr.from_dict(test_novel_mi_s)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr1,100,1600,+,transcript,id_nov_gene_1,nov_gene_1_tx_mi_s_p,nov_gene_1,NaN,protein_coding
1,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_s_p,nov_gene_1,1.0,protein_coding
2,chr1,1000,1200,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_s_p,nov_gene_1,2.0,protein_coding
3,chr1,1400,1600,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_s_p,nov_gene_1,3.0,protein_coding
4,chr1,100,1600,+,transcript,id_nov_gene_1,nov_gene_1_tx_mi_s_f,nov_gene_1,NaN,protein_coding
5,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_s_f,nov_gene_1,1.0,protein_coding
6,chr1,1000,1300,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_s_f,nov_gene_1,2.0,protein_coding
7,chr1,1400,1600,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_s_f,nov_gene_1,3.0,protein_coding
8,chr1,100,1850,+,transcript,id_nov_gene_1,nov_gene_1_tx_mi_s_no_pas_no_motif_f,nov_gene_1,NaN,protein_coding
9,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_s_no_pas_no_motif_f,nov_gene_1,1.0,protein_coding


In [11]:
# 3. Internal intron bleedthrough (e.g. SIN3B)
# For these events to pass, the 5'end of the last exon matches the 5'end of the overlapping ref exon

test_novel_mi_ext = {"Chromosome": ["chr1"]*12,
                "Start": [100,100,1000,2000] + [100,100,1000,1950] + [100,100,1000,2000],
                "End": [2400,300,1200,2400] + [2400,300,1200,2400] + [2600,300,1200,2600],
                "Strand": ["+"]*12,
                "Feature": (["transcript"] + ["exon"]*3)*3,
                "gene_id": ["id_nov_gene_1"]*12,
                "transcript_id": ["nov_gene_1_tx_mi_ext_p"]*4 + ["nov_gene_1_tx_mi_ext_f"]*4 + ["nov_gene_1_tx_mi_ext_no_pas_motif_p"]*4,
                     "gene_name": ["nov_gene_1"]*12,
                "exon_number": [None,1,2,3]*3,
                 "gene_type": ["protein_coding"] * 12
               }

test_novel_trs.append(pr.from_dict(test_novel_mi_ext))
update_pas_ends(pas_ends, "chr1", 2399, 2400, "+")
pr.from_dict(test_novel_mi_ext)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr1,100,2400,+,transcript,id_nov_gene_1,nov_gene_1_tx_mi_ext_p,nov_gene_1,NaN,protein_coding
1,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_ext_p,nov_gene_1,1.0,protein_coding
2,chr1,1000,1200,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_ext_p,nov_gene_1,2.0,protein_coding
3,chr1,2000,2400,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_ext_p,nov_gene_1,3.0,protein_coding
4,chr1,100,2400,+,transcript,id_nov_gene_1,nov_gene_1_tx_mi_ext_f,nov_gene_1,NaN,protein_coding
5,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_ext_f,nov_gene_1,1.0,protein_coding
6,chr1,1000,1200,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_ext_f,nov_gene_1,2.0,protein_coding
7,chr1,1950,2400,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_ext_f,nov_gene_1,3.0,protein_coding
8,chr1,100,2600,+,transcript,id_nov_gene_1,nov_gene_1_tx_mi_ext_no_pas_motif_p,nov_gene_1,NaN,protein_coding
9,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_ext_no_pas_motif_p,nov_gene_1,1.0,protein_coding


In [12]:
pas_ends

{'Chromosome': ['chr1', 'chr2', 'chr1', 'chr1'],
 'Start': [699, 880, 1599, 2399],
 'End': [700, 881, 1600, 2400],
 'Strand': ['+', '-', '+', '+']}

In [13]:
#4. Last exon spliced (same rules as prev spliced apply - for simplicity will just make 1 that passes)
test_novel_li_spl = {"Chromosome": ["chr1"]*5,
                "Start": [100,100,1000,2000,3800],
                "End": [3800,300,1200,2200,4000],
                "Strand": ["+"]*5,
                "Feature": ["transcript"] + ["exon"]*4,
                "gene_id": ["id_nov_gene_1"]*5,
                "transcript_id": ["nov_gene_1_li_spl_p"]*5,
                     "gene_name": ["nov_gene_1"]*5,
                       "exon_number": [None,1,2,3,4],
                       "gene_type": ["protein_coding"] * 5
               }

test_novel_trs.append(pr.from_dict(test_novel_li_spl))
update_pas_ends(pas_ends, "chr1", 3999,4000, "+")
pr.from_dict(test_novel_li_spl)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr1,100,3800,+,transcript,id_nov_gene_1,nov_gene_1_li_spl_p,nov_gene_1,NaN,protein_coding
1,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_li_spl_p,nov_gene_1,1.0,protein_coding
2,chr1,1000,1200,+,exon,id_nov_gene_1,nov_gene_1_li_spl_p,nov_gene_1,2.0,protein_coding
3,chr1,2000,2200,+,exon,id_nov_gene_1,nov_gene_1_li_spl_p,nov_gene_1,3.0,protein_coding
4,chr1,3800,4000,+,exon,id_nov_gene_1,nov_gene_1_li_spl_p,nov_gene_1,4.0,protein_coding


In [14]:
#5. Last exon extension (same rules as prev extensions apply - for simplicity will just make 1 that passes)
test_novel_le_ext_plus = {"Chromosome": ["chr1"]*5,
                "Start": [1000,100,1000,2000,3000],
                "End": [3600,300,1200,2200,3600],
                "Strand": ["+"]*5,
                "Feature": ["transcript"] + ["exon"]*4,
                "gene_id": ["id_nov_gene_1"]*5,
                "transcript_id": ["nov_gene_1_novel_tx_le_ext_plus_p"] * 5,
                          "gene_name": ["nov_gene_1"]*5,
                "exon_number": [None,1,2,3,4],
                           "gene_type": ["protein_coding"] * 5}

test_novel_trs.append(pr.from_dict(test_novel_le_ext_plus))
update_pas_ends(pas_ends, "chr1", 3599, 3600, "+")
pr.from_dict(test_novel_le_ext_plus)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr1,1000,3600,+,transcript,id_nov_gene_1,nov_gene_1_novel_tx_le_ext_plus_p,nov_gene_1,NaN,protein_coding
1,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_novel_tx_le_ext_plus_p,nov_gene_1,1.0,protein_coding
2,chr1,1000,1200,+,exon,id_nov_gene_1,nov_gene_1_novel_tx_le_ext_plus_p,nov_gene_1,2.0,protein_coding
3,chr1,2000,2200,+,exon,id_nov_gene_1,nov_gene_1_novel_tx_le_ext_plus_p,nov_gene_1,3.0,protein_coding
4,chr1,3000,3600,+,exon,id_nov_gene_1,nov_gene_1_novel_tx_le_ext_plus_p,nov_gene_1,4.0,protein_coding


In [15]:
# last exon extension on the minus strand
# Can only get it to fail the distance cut-off as not enough 'genomic space' for a 100nt extension
test_novel_le_ext_minus = {"Chromosome": ["chr2"]*4,
                "Start": [50,50,700,1000],
                "End": [1200,200,800,1200],
                "Strand": ["-"]*4,
                "Feature": ["transcript"] + ["exon"]*3,
                "gene_id": ["id_nov_gene_2"]*4,
                "transcript_id": ["nov_gene_2_tx_le_ext_minus_f"]*4,
                           "gene_name": ["nov_gene_2"]*4,
                "exon_number": [None,3,2,1],
                             "gene_type": ["protein_coding"] * 4
               }

test_novel_trs.append(pr.from_dict(test_novel_le_ext_minus))
pr.from_dict(test_novel_le_ext_minus)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr2,50,1200,-,transcript,id_nov_gene_2,nov_gene_2_tx_le_ext_minus_f,nov_gene_2,NaN,protein_coding
1,chr2,50,200,-,exon,id_nov_gene_2,nov_gene_2_tx_le_ext_minus_f,nov_gene_2,3.0,protein_coding
2,chr2,700,800,-,exon,id_nov_gene_2,nov_gene_2_tx_le_ext_minus_f,nov_gene_2,2.0,protein_coding
3,chr2,1000,1200,-,exon,id_nov_gene_2,nov_gene_2_tx_le_ext_minus_f,nov_gene_2,1.0,protein_coding


In [16]:
# 1 last transcript on the minus strand that contains a PAS motif
# (to check that motif matching on the opposite strand works correctly)
test_novel_mi_spl_minus = {"Chromosome": ["chr2"]*4,
                "Start": [400,400,700,1000],
                "End": [1200,550,800,1200],
                "Strand": ["-"]*4,
                "Feature": ["transcript"] + ["exon"]*3,
                "gene_id": ["id_nov_gene_2"]*4,
                "transcript_id": ["nov_gene_2_tx_mi_spl_minus_no_pas_motif_p"]*4,
                           "gene_name": ["nov_gene_2"]*4,
                "exon_number": [None,3,2,1],
                             "gene_type": ["protein_coding"] * 4
               }
test_novel_trs.append(pr.from_dict(test_novel_mi_spl_minus))
pr.from_dict(test_novel_mi_spl_minus)

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr2,400,1200,-,transcript,id_nov_gene_2,nov_gene_2_tx_mi_spl_minus_no_pas_motif_p,nov_gene_2,NaN,protein_coding
1,chr2,400,550,-,exon,id_nov_gene_2,nov_gene_2_tx_mi_spl_minus_no_pas_motif_p,nov_gene_2,3.0,protein_coding
2,chr2,700,800,-,exon,id_nov_gene_2,nov_gene_2_tx_mi_spl_minus_no_pas_motif_p,nov_gene_2,2.0,protein_coding
3,chr2,1000,1200,-,exon,id_nov_gene_2,nov_gene_2_tx_mi_spl_minus_no_pas_motif_p,nov_gene_2,1.0,protein_coding


In [17]:
test_novel = pr.concat(test_novel_trs)
test_novel

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr1,100,700,+,transcript,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,NaN,protein_coding
1,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,1.0,protein_coding
2,chr1,500,700,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,2.0,protein_coding
3,chr1,100,700,+,transcript,id_nov_gene_1,nov_gene_1_tx_fi_f,nov_gene_1,NaN,protein_coding
4,chr1,100,350,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_f,nov_gene_1,1.0,protein_coding
5,chr1,500,700,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_f,nov_gene_1,2.0,protein_coding
6,chr1,100,1600,+,transcript,id_nov_gene_1,nov_gene_1_tx_mi_s_p,nov_gene_1,NaN,protein_coding
7,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_s_p,nov_gene_1,1.0,protein_coding
8,chr1,1000,1200,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_s_p,nov_gene_1,2.0,protein_coding
9,chr1,1400,1600,+,exon,id_nov_gene_1,nov_gene_1_tx_mi_s_p,nov_gene_1,3.0,protein_coding


In [18]:
test_trs = pr.concat([test_novel, test_ref])
test_trs

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type
0,chr1,100,700,+,transcript,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,NaN,protein_coding
1,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,1.0,protein_coding
2,chr1,500,700,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,2.0,protein_coding
3,chr1,100,700,+,transcript,id_nov_gene_1,nov_gene_1_tx_fi_f,nov_gene_1,NaN,protein_coding
4,chr1,100,350,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_f,nov_gene_1,1.0,protein_coding
...,...,...,...,...,...,...,...,...,...,...
56,chr2,1000,1200,-,exon,id_nov_gene_2,nov_gene_2_tx_mi_spl_minus_no_pas_motif_p,nov_gene_2,1.0,protein_coding
57,chr2,100,1200,-,transcript,id_ref_gene_2,ref_g2_tr_1,ref_gene_2,NaN,protein_coding
58,chr2,100,200,-,exon,id_ref_gene_2,ref_g2_tr_1,ref_gene_2,3.0,protein_coding
59,chr2,700,800,-,exon,id_ref_gene_2,ref_g2_tr_1,ref_gene_2,2.0,protein_coding


In [19]:
test_trs.Source = "PAPA_tests"
test_trs

,Chromosome,Start,End,Strand,Feature,gene_id,transcript_id,gene_name,exon_number,gene_type,Source
0,chr1,100,700,+,transcript,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,NaN,protein_coding,PAPA_tests
1,chr1,100,300,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,1.0,protein_coding,PAPA_tests
2,chr1,500,700,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_p,nov_gene_1,2.0,protein_coding,PAPA_tests
3,chr1,100,700,+,transcript,id_nov_gene_1,nov_gene_1_tx_fi_f,nov_gene_1,NaN,protein_coding,PAPA_tests
4,chr1,100,350,+,exon,id_nov_gene_1,nov_gene_1_tx_fi_f,nov_gene_1,1.0,protein_coding,PAPA_tests
...,...,...,...,...,...,...,...,...,...,...,...
56,chr2,1000,1200,-,exon,id_nov_gene_2,nov_gene_2_tx_mi_spl_minus_no_pas_motif_p,nov_gene_2,1.0,protein_coding,PAPA_tests
57,chr2,100,1200,-,transcript,id_ref_gene_2,ref_g2_tr_1,ref_gene_2,NaN,protein_coding,PAPA_tests
58,chr2,100,200,-,exon,id_ref_gene_2,ref_g2_tr_1,ref_gene_2,3.0,protein_coding,PAPA_tests
59,chr2,700,800,-,exon,id_ref_gene_2,ref_g2_tr_1,ref_gene_2,2.0,protein_coding,PAPA_tests


In [20]:
test_novel.to_gtf("../tests/test_annotations/test_transcripts_novel.gtf")
test_ref.to_gtf("../tests/test_annotations/test_transcripts_reference.gtf")
test_trs.to_gtf("../tests/test_annotations/test_transcripts_combined.gtf")

## Generate fake reference genome for test transcripts


In [21]:
# Each chrom will start at 0 - from there on need to define a large enough range for the region to get trs + a bit of intergenic sequence
test_trs.as_df().groupby("Chromosome")["End"].max()


Chromosome
chr1    4000
chr2    1500
Name: End, dtype: int32

In [22]:
# For simplicity let's make both chromosomes 10kb -
# Need to generate random sequences for each chromosome
random.seed(123)
chr1 = "".join(random.choices("ATCG", k=10000))
chr2 = "".join(random.choices("ATCG", k=10000))

seq_chr1 = SeqRecord(seq=MutableSeq(chr1), id="chr1", name="chr1")
seq_chr2 = SeqRecord(seq=MutableSeq(chr2), id="chr2", name="chr2")

seq_chr1

SeqRecord(seq=MutableSeq('AATAGACTGATTAATACATTGAAGAGCTGGTGACCGTTGCCCCGTCTAGAGGCA...GAG'), id='chr1', name='chr1', description='<unknown description>', dbxrefs=[])

Now need to make sure specific last exons do / do not contain PAS motifs

**To contain motifs**
- nov_gene_2_tx_mi_spl_minus_no_pas_motif_p
- nov_gene_1_tx_mi_ext_no_pas_motif_p

**To not contain motifs**
- nov_gene_1_tx_mi_s_no_pas_no_motif_f

For each of these, need to extract last exons & check the corresponding sequence does not contain any PAS motifs

In [23]:
gruber_pas_motifs = """AATAAA
ATTAAA
TATAAA
AGTAAA
AATACA
CATAAA
AATATA
GATAAA
AATGAA
AAGAAA
ACTAAA
AATAGA
AATAAT
AACAAA
ATTACA
ATTATA
AACAAG
AATAAG""".split("\n")

# Generate motifs object
# from generate a list of Seq objects storing motifs
pas_seqs = [Seq(motif) for motif in gruber_pas_motifs]
pas_motifs = motifs.create(pas_seqs)

In [24]:
# x = Seq(seq_chr2.seq[400:500])
# print(x)
# print(Seq.reverse_complement(x))

In [25]:
# for idx, motif in pas_motifs.instances.search(Seq(seq_chr2.seq[400:500]).reverse_complement()):
#     print(idx)
#     print(motif)

In [26]:
# These transcripts should have PAS motifs within the final 100bp
tx_w_motifs = ["nov_gene_1_tx_mi_ext_no_pas_motif_p", "nov_gene_2_tx_mi_spl_minus_no_pas_motif_p"]


tes_tx_w_motifs = get_terminal_regions(test_novel.subset(lambda df: (df["Feature"] == "exon") & (df["transcript_id"].isin(tx_w_motifs))))
print(tes_tx_w_motifs)

for k, v in tes_tx_w_motifs:
    for idx, row in v.iterrows():
        if row["Strand"] == "+":
            reg_start = row["End"] - 100
            reg_end = row["End"]
            reg_str = row["Strand"]
        
        else:
            reg_start = row["Start"]
            reg_end = row["Start"] + 100
            reg_str = row["Strand"]
            
        if k[0] == "chr1":
            if reg_str == "+":
                found_motifs = ";".join("_".join([str(idx), str(motif)]) for idx, motif in pas_motifs.instances.search(seq_chr1.seq[reg_start:reg_end]))
            
            else:
                reg_seq = Seq(seq_chr2.seq[reg_start:reg_end]).reverse_complement()                
                found_motifs = ";".join("_".join([str(idx), str(motif)]) for idx, motif in pas_motifs.instances.search(Seq(seq_chr1.seq[reg_start:reg_end]).reverse_complement()))
            
        else:
            if reg_str == "+":
                found_motifs = ";".join("_".join([str(idx), str(motif)]) for idx, motif in pas_motifs.instances.search(seq_chr2.seq[reg_start:reg_end]))
            
            else:
                #x = "_".join(idx, motif for idx, motif in pas_motifs.instances.search(Seq.reverse_complement(Seq(seq_chr2.seq[reg_start:reg_end]))))
                #print(x)    
                reg_seq = Seq(seq_chr2.seq[reg_start:reg_end]).reverse_complement()
                found_motifs = ";".join("_".join([str(idx), str(motif)]) for idx, motif in pas_motifs.instances.search(reg_seq))
            
        
        print(f"{row['transcript_id']} - found motifs - {found_motifs}")

# first check if by random chance the motif has 
#";".join("_".join(idx, motif) for idx, motif in pas_motifs.instances.search(seq_chr1.seq[2600-100:2600]))




+--------------+-----------+-----------+--------------+------------+---------------+-------------------------------------------+-------+
| Chromosome   |     Start |       End | Strand       | Feature    | gene_id       | transcript_id                             | +3    |
| (category)   |   (int32) |   (int32) | (category)   | (object)   | (object)      | (object)                                  | ...   |
|--------------+-----------+-----------+--------------+------------+---------------+-------------------------------------------+-------|
| chr1         |      2000 |      2600 | +            | exon       | id_nov_gene_1 | nov_gene_1_tx_mi_ext_no_pas_motif_p       | ...   |
| chr2         |       400 |       550 | -            | exon       | id_nov_gene_2 | nov_gene_2_tx_mi_spl_minus_no_pas_motif_p | ...   |
+--------------+-----------+-----------+--------------+------------+---------------+-------------------------------------------+-------+
Stranded PyRanges object has 2 rows and 1

In [27]:
# minus strand tx -nov_gene_2_tx_mi_spl_minus_no_pas_motif_p - has a PAS motif within last 100bp
# need to add motif for nov_gene_1_tx_mi_ext_no_pas_motif_p
mut_tx_te = tes_tx_w_motifs.subset(lambda df: df["transcript_id"] == "nov_gene_1_tx_mi_ext_no_pas_motif_p")
# Set seq to mutate to ~ 25 bp upstream of end
mut_reg_start = mut_tx_te.three_end().End[0] - 26
mut_reg_end = mut_tx_te.three_end().End[0] - 20
mut_reg_end - mut_reg_start

seq_chr1.seq[mut_reg_start:mut_reg_end] = "AATAAA"

seq_chr1.seq[mut_reg_start:mut_reg_end]

MutableSeq('AATAAA')

In [28]:
# double check it has a motif
(";".join("_".join([str(idx), str(motif)]) 
          for idx, motif in pas_motifs.instances.search(seq_chr1.seq[mut_tx_te.End.iloc[0] - 100:mut_tx_te.End.iloc[0]])))

'74_AATAAA'

In [29]:
# now check this tx - nov_gene_1_tx_mi_s_no_pas_no_motif_f - doesn't have any PAS motifs
tx_w_no_motifs = ["nov_gene_1_tx_mi_s_no_pas_no_motif_f"]

tes_tx_w_no_motifs = get_terminal_regions(test_novel.subset(lambda df: (df["Feature"] == "exon") & (df["transcript_id"].isin(tx_w_no_motifs))))
print(tes_tx_w_no_motifs)

for k, v in tes_tx_w_no_motifs:
    for idx, row in v.iterrows():
        if row["Strand"] == "+":
            reg_start = row["End"] - 100
            reg_end = row["End"]
            reg_str = row["Strand"]
        
        else:
            reg_start = row["Start"]
            reg_end = row["Start"] + 100
            reg_str = row["Strand"]
            
        if k[0] == "chr1":
            if reg_str == "+":
                found_motifs = ";".join("_".join([str(idx), str(motif)]) for idx, motif in pas_motifs.instances.search(seq_chr1.seq[reg_start:reg_end]))
            
            else:
                reg_seq = Seq(seq_chr2.seq[reg_start:reg_end]).reverse_complement()                
                found_motifs = ";".join("_".join([str(idx), str(motif)]) for idx, motif in pas_motifs.instances.search(Seq(seq_chr1.seq[reg_start:reg_end]).reverse_complement()))
            
        else:
            if reg_str == "+":
                found_motifs = ";".join("_".join([str(idx), str(motif)]) for idx, motif in pas_motifs.instances.search(seq_chr2.seq[reg_start:reg_end]))
            
            else:
                #x = "_".join(idx, motif for idx, motif in pas_motifs.instances.search(Seq.reverse_complement(Seq(seq_chr2.seq[reg_start:reg_end]))))
                #print(x)    
                reg_seq = Seq(seq_chr2.seq[reg_start:reg_end]).reverse_complement()
                found_motifs = ";".join("_".join([str(idx), str(motif)]) for idx, motif in pas_motifs.instances.search(reg_seq))
            
        
        print(f"{row['transcript_id']} - found motifs - {found_motifs}")


+--------------+-----------+-----------+--------------+------------+---------------+--------------------------------------+-------------+-------+
| Chromosome   |     Start |       End | Strand       | Feature    | gene_id       | transcript_id                        | gene_name   | +2    |
| (category)   |   (int32) |   (int32) | (category)   | (object)   | (object)      | (object)                             | (object)    | ...   |
|--------------+-----------+-----------+--------------+------------+---------------+--------------------------------------+-------------+-------|
| chr1         |      1750 |      1850 | +            | exon       | id_nov_gene_1 | nov_gene_1_tx_mi_s_no_pas_no_motif_f | nov_gene_1  | ...   |
+--------------+-----------+-----------+--------------+------------+---------------+--------------------------------------+-------------+-------+
Stranded PyRanges object has 1 rows and 10 columns from 1 chromosomes.
For printing, the PyRanges was sorted on Chromosome a

In [30]:
with open("../tests/test_annotations/test_genome.fa", "w") as out_fasta:
    for seq in [seq_chr1, seq_chr2]:
        SeqIO.write(seq,out_fasta, "fasta")
        

In [31]:
# Now need to make transcripts FASTA
os.system("which gffread")

0

In [32]:
gtf = "../tests/test_annotations/test_transcripts_combined.gtf"
genome_fa = "../tests/test_annotations/test_genome.fa"
out_tx_fa = "../tests/test_annotations/test_transcripts.fa"

os.system(f"gffread -w {out_tx_fa} -g {genome_fa} {gtf}")

0

### Generate BED file of PolyASites

txipts that pass all isoform filters but don't havr he 'no_pas_motif' annotation (i.e. don't have an annotated PAS but pass due to motif)

In [41]:
pas_bed = (test_novel.subset(lambda df: df.transcript_id.str.endswith("_p"))
 .subset(lambda df: ~df.transcript_id.str.contains("no_pas_motif", regex=False))
 .subset(lambda df: df["Feature"] == "transcript")
 .three_end() # last templated coord as the rep for the PAS
 .assign("Name", lambda df: df["Chromosome"].str.cat(df[["End", "Strand"]].astype(str), sep=":"))
 [["Name"]]
 )

pas_bed

,Chromosome,Start,End,Strand,Name
0,chr1,699,700,+,chr1:700:+
1,chr1,1599,1600,+,chr1:1600:+
2,chr1,2399,2400,+,chr1:2400:+
3,chr1,3799,3800,+,chr1:3800:+
4,chr1,3599,3600,+,chr1:3600:+
5,chr2,880,881,-,chr2:881:-


In [42]:
pas_bed.to_bed("../tests/test_annotations/test_pas.bed")